In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencc-python-reimplemented fuzzywuzzy python-Levenshtein


In [ ]:
import re


In [ ]:
import pandas as pd
from opencc import OpenCC
from fuzzywuzzy import process

# 初始化简繁转换
cc = OpenCC('t2s')

# 读取表格
df_a = pd.read_excel("/content/drive/MyDrive/副本李白情感比例200首 (整理中...1117.xlsx")
df_b = pd.read_excel("/content/drive/MyDrive/李白繁体作品.xlsx")  # 繁体全集

print("简体表格的列名：", df_a.columns.tolist())
print("繁体表格的列名：", df_b.columns.tolist())


简体表格的列名： ['年份', '地点', '作品名（简体）', '作品名（繁体）', '诗歌内容（繁体）', '东经', '北纬', 'text_full', '豪放与激昂', '喜悦与欢乐', '哀怨与悲伤', '思乡与怀古', '孤寂与落寞', '友情与知己', 'top1_emotion', 'top2_emotion', 'top3_emotion', 'top1_weight', 'top2_weight', 'top3_weight']
繁体表格的列名： ['作品名（繁体）', '诗歌内容（繁体）']


In [ ]:
def match_title(title):
    match, score = process.extractOne(title, df_b["作品名_简体"])
    return match, score

df_a["匹配结果"], df_a["匹配分数"] = zip(*df_a["作品名（简体）"].apply(match_title))


In [ ]:

#为表添加“简体对照列”
df_b["作品名_简体"] = df_b["作品名（繁体）"].apply(lambda x: cc.convert(str(x)))

#拆分表A中多地点、多作品列（逗号、顿号、分号等分隔）
def smart_split(cell):
    if pd.isna(cell): return []
    parts = re.split(r'[、,，；;、\s]+', str(cell))
    return [p.strip() for p in parts if p.strip()]

records = []
for _, row in df_a.iterrows():
    year = str(row["年份"]).strip()
    locs = smart_split(row["地点"])
    poems = smart_split(row["作品名（简体）"])

    # 若地点数少于作品数，则沿用最后一个地点
    for i, poem in enumerate(poems):
        loc = locs[min(i, len(locs)-1)] if locs else "未知"
        records.append({"年份": year, "地点": loc, "作品名（简体）": poem})

df_flat = pd.DataFrame(records)
print(f"✅ 已展开为 {len(df_flat)} 行。")

#模糊匹配函数
def match_poem(poem_name, candidates, threshold=80):
    match, score = process.extractOne(poem_name, candidates)
    return match if score >= threshold else None

# 匹配简体→繁体全集
matched = []
for _, row in df_flat.iterrows():
    simp = row["作品名（简体）"]
    match_name = match_poem(simp, df_b["作品名_简体"].tolist())

    if match_name:
        match_row = df_b[df_b["作品名_简体"] == match_name].iloc[0]
        matched.append({
            "年份": row["年份"],
            "地点": row["地点"],
            "作品名（简体）": simp,
            "作品名（繁体）": match_row["作品名（繁体）"],
            "诗歌内容（繁体）": match_row["诗歌内容（繁体）"]
        })
    else:
        matched.append({
            "年份": row["年份"],
            "地点": row["地点"],
            "作品名（简体）": simp,
            "作品名（繁体）": "未匹配",
            "诗歌内容（繁体）": "未找到"
        })

df_final = pd.DataFrame(matched)

#匹配统计
matched_count = (df_final["作品名（繁体）"] != "未匹配").sum()
total = len(df_final)
print(f"✅ 成功匹配 {matched_count}/{total} 首诗，匹配率 {matched_count/total:.1%}")

#保存结果
df_final.to_excel("/content/li_bai_merged.xlsx", index=False)
print("✅ 已保存合并结果到 /content/li_bai_merged-2.xlsx")


✅ 已展开为 919 行。


✅ 成功匹配 856/919 首诗，匹配率 93.1%
✅ 已保存合并结果到 /content/li_bai_merged-2.xlsx


In [ ]:
!pip install opencc-python-reimplemented


In [ ]:
import pandas as pd


EXCEL_PATH = "/content/drive/MyDrive/fantimatch-2v2v2.xlsx"
OUTPUT_PATH = "/content/fantimatch-2v2-4.xlsx"


df = pd.read_excel(EXCEL_PATH)
df.columns = df.columns.str.strip()
print(f"✅ 成功读取 {len(df)} 条记录，列名为：{df.columns.tolist()}")

# 地点-经纬度映射表
#   格式：'地点名': (纬度, 经度)
coords_map = {
    "绵州": (104.76, 31.48), "大匡山": (104.85, 31.75), "成都": (104.07, 30.67),
    "峨眉": (103.48, 29.56), "渝州": (106.55, 29.56), "江陵": (112.24, 30.35),
    "峨眉山": (103.48, 29.56), "皖公山": (117.03, 30.63), "荆门山": (111.88, 30.08),
    "广陵": (119.42, 32.39), "扬州": (119.42, 32.39), "越州": (120.58, 30.01),
    "苏州": (120.62, 31.32), "杭州": (120.16, 30.27), "金陵": (118.78, 32.04),
    "台州": (121.42, 28.67), "白兆山": (113.68, 31.28), "终南山": (108.95, 34.05),
    "邠州": (108.17, 35.34), "坊州": (109.11, 35.54), "嵩山": (113.05, 34.49),
    "颖阳": (113.18, 34.42), "洛阳": (112.45, 34.62), "襄阳": (112.14, 32.08),
    "恒山": (113.46, 39.75), "宋城": (115.65, 34.44), "楚州": (119.16, 33.59),
    "巴陵": (113.09, 29.37), "徂徕山": (117.27, 36.09), "泰安": (117.16, 36.26),
    "梁宋": (115.45, 34.57), "齐州": (117.00, 36.67), "曲阜": (116.98, 35.59),
    "济宁": (116.59, 35.41), "江淮": (119.00, 32.50), "庐山": (115.98, 29.71),
    "幽州": (116.46, 39.92), "广平郡": (114.87, 36.68), "敬亭山": (118.75, 30.95),
    "宣城": (118.75, 30.95), "泾县": (118.41, 30.68), "秋浦": (117.48, 30.69),
    "寻阳": (116.00, 29.74), "襄州": (112.14, 32.08), "白帝城": (109.69, 31.09),
    "岳州": (113.09, 29.37), "零陵": (111.69, 26.23),"安陆": (113.75, 31.43),
    "雁门关":(112.86,39.18), "当涂":(118.48,31.55),"安州北寿山":(113.75, 31.43),
    "陈州": (114.87,33.73), "枣庄":(117.32,34.81),"长安":(108.96,34.22),
    "南京（江宁）":(118.84,31.95),"池州（秋浦）":(117.49,30.66),"金乡":(116.31,35.07),"兖州（瑕丘）":(116.78,35.55),"兖州":(116.78,35.55),
    "单县（单父）":(116.07,34.79), "单父":(116.07,34.79),"九江（寻阳）": (116.00, 29.74),"汾阳（西河）":(111.76,37.27),"魏县（贵乡）":(116.94,36.36),
    "永年（临洺）":(114.54,36.64),"宜昌（夷陵）":(111.32,30.76),"巫山":(109.87,30.07),
    "奉节":(109.46,31.02),"武汉（江夏）":(114.32,30.38),"武汉（江夏）":(114.32,30.38),"临湘":(113.44,29.47),"岳阳（巴陵）":(113.11,29.37),
    "永州（零陵）":(111.61,26.22),"宜兴（义兴）":(119.82,31.36),"和县（历阳）":(118.35,31.72),"溧阳":(119.08,31.09),
    "商州": (109.9284, 33.8659),"杞县（雍丘）": (114.7692, 34.5543),"砀山": (116.324, 34.41),
    "聊城（博州）": (115.98, 36.45),"吴桥（安陵）": (116.3858, 37.6260),"深州": (115.52, 37.63),
    "兖州（瑕丘）": (116.78, 35.55),"金乡": (116.31, 35.07),"武汉":(114.32,30.38),
    "叶县": (113.35, 33.62),"开封": (114.30, 34.79),"滑县（白马）": (114.51, 35.57),
    "安阳（邺郡）": (114.35, 36.10),  "广平（清漳）": (114.87, 36.68),"邯郸": (114.482, 36.61),
    "定陶（曹州）": (115.59, 35.14),"青阳": (117.85, 30.63),"南陵": (118.33, 30.92),
    "丹阳": (119.57, 32.00),"潜山": (116.33, 30.63),"太湖": (115.45, 30.28),"宿松": (116.11, 30.15),
    "襄阳":(112.12,32.00),"无锡": (120.30, 31.55),"湖州": (120.09, 30.89),"绍兴": (120.59, 29.99),"天台": (121.03, 29.14),
    "庐江（舒城）": (116.94, 31.47),"大兴（范阳）": (115.97, 39.00),"武功（太白山）": (108.21, 34.26),
    "岐山": (107.60, 34.33),"彬县（新平）": (108.08, 35.04),"应城": (113.57, 30.95),"嵊州（剡县）": (120.80, 29.58),
    "宜昌": (112.20, 30.33),"四川江油": (104.73, 31.78),"平武": (104.50, 32.42),"南阳": (112.53, 33.00),
    "黄陵（中部）": (109.26, 35.58),"南昌（豫章）": (115.86, 28.68),"黄石": (115.04, 30.20),
    "亳州（谯郡）": (115.78, 33.85),"任城": (116.60, 35.41),"安宜": (119.31, 33.24),
    "下邳": (117.57, 33.77),"淮阴": (119.03, 33.63),"天门山": (110.48, 29.05),"重庆": (106.55, 29.55),
    "怀远（钟离）": (117.38, 32.92),"定陶": (115.59, 35.14),"商丘":(115.65, 34.42)
}


def get_coords_by_name(place):
    """根据地点名查找映射字典"""
    if pd.isna(place):
        return None, None
    name = str(place).strip()
    for key, (lat, lon) in coords_map.items():
        if key in name:  # 支持部分匹配
            return lat, lon
    return None, None  # 没匹配到返回空

#添加经纬度
df["东经"], df["北纬"] = zip(*df["地点"].map(get_coords_by_name))


#统计和输出
print("\n📊 匹配统计：")
matched = df["东经"].notna().sum()
print(f"成功匹配到坐标的地点数：{matched}/{len(df)}")

print("\n✅ 前几行预览：")
print(df.head(10))

# 保存结果
df.to_excel(OUTPUT_PATH, index=False)
print(f"\n📁 已保存结果到：{OUTPUT_PATH}")


✅ 成功读取 861 条记录，列名为：['年份', '地点', '作品名（简体）', '作品名（繁体）', '诗歌内容（繁体）', '东经', '北纬']

📊 匹配统计：
成功匹配到坐标的地点数：861/861

✅ 前几行预览：
    年份    地点   作品名（简体）   作品名（繁体）  \
0  715  四川江油      《初月》        初月   
1  715  四川江油    《雨后望月》      雨後望月   
2  715  四川江油      《对雨》        對雨   
3  715  四川江油      《晓晴》        曉晴   
4  715  四川江油     《望夫石》       望夫石   
5  718    平武   《赠江油尉厅》      贈江油尉   
6  718  四川江油  《寻雍尊师隐居》    尋雍尊師隱居   
7  719  四川江油  访戴天山道士不遇  訪戴天山道士不遇   
8  720    成都  《登锦城散花楼》    登錦城散花樓   
9  720    成都      《春感》       春感詩   

                                            诗歌内容（繁体）      东经     北纬  
0   玉蟾离海上，白露湿花时。云畔风生爪，沙头水浸眉。乐哉弦管客，愁杀战征儿。因绝西园赏，临风一咏诗。  104.73  31.78  
1   四郊陰靄散，開戶半蟾生。萬里舒霜合，一條江練橫。出時山眼白，高後海心明。爲惜如團扇，長吟到五更。  104.73  31.78  
2   卷帘聊举目，露湿草绵芊。古岫藏云毳，空庭织碎烟。水纹愁不起，风线重难牵。尽日扶犁叟，往来江树前。  104.73  31.78  
3   野凉疏雨歇，春色遍萋萋。鱼跃青池满，莺吟绿树低。野花妆面湿，山草纽斜齐。零落残雪片，风吹挂竹溪。  104.73  31.78  
4   髣髴古容仪，含愁带曙辉。露如今日泪，苔似昔年衣。有恨同湘女，无言类楚妃。寂然芳霭内，犹若待夫归。  104.73  31.78  
5   岚光深院里，傍砌水泠泠。野燕巢官舍，溪云入古厅。日斜孤吏过，帘捲乱峰青。五色神仙尉，焚香读道经。  104.50  32.4